In [1]:
import pandas as pd
import json
import requests
from tqdm.autonotebook import tqdm
from dms2dec.dms_convert import dms2dec

In [38]:
CLIENT_ID = 'EG4MYYIKUQNIXZPRW3TKSHYP5PG2AGCCJ01B1KPUYKTPEVRP' # your Foursquare ID
CLIENT_SECRET = '4RIINJ1N4BICVMLX3LAR5NBWET21BZTCX54SZ1NJNEDGBIIR' # your Foursquare Secret
VERSION = '20210201' # Foursquare API version

In [18]:
#venue_search_url = f"https://api.foursquare.com/v2/venues/search?client_id={CLIENT_ID}&client_secret={CLIENT_SECRET}&v={VERSION}"
venue_explore_url = f"https://api.foursquare.com/v2/venues/explore?client_id={CLIENT_ID}&client_secret={CLIENT_SECRET}&v={VERSION}"

In [19]:
suburb = pd.read_csv('data/suburb.csv')

In [20]:
suburb = suburb[~suburb['latitude'].isna()]

In [21]:
suburb

,href,latitude,longitude,suburb
4,/wiki/Blue_Mountains_National_Park,33°37′S,150°28′E,Blue Mountains National Park
6,/wiki/Sydney,33°51′54″S,151°12′34″E,Sydney
7,/wiki/City_of_Penrith,33°45′S,150°42′E,City of Penrith
8,/wiki/Camden_Council_(New_South_Wales),34°03′S,150°42′E,Camden Council
9,/wiki/City_of_Campbelltown_(New_South_Wales),34°04′S,150°49′E,City of Campbelltown
...,...,...,...,...
726,/wiki/City_of_Hawkesbury,33°25′S,150°47′E,Hawkesbury
727,/wiki/The_Hills_Shire,33°46′S,151°00′E,The Hills
728,/wiki/Municipality_of_Hunter%27s_Hill,33°49′S,151°08′E,Hunter's Hill
729,/wiki/Ku-ring-gai_Council,33°45′15″S,151°09′06″E,Ku-ring-gai


In [22]:
def convert_ll(row):
    row['lat_dec'] = dms2dec(row['latitude'])
    row['lng_dec'] = dms2dec(row['longitude'])
    return row

In [23]:
suburb = suburb.apply(convert_ll, axis=1)

In [24]:
suburb

,href,latitude,longitude,suburb,lat_dec,lng_dec
4,/wiki/Blue_Mountains_National_Park,33°37′S,150°28′E,Blue Mountains National Park,-33.616667,150.466667
6,/wiki/Sydney,33°51′54″S,151°12′34″E,Sydney,-33.865000,151.209444
7,/wiki/City_of_Penrith,33°45′S,150°42′E,City of Penrith,-33.750000,150.700000
8,/wiki/Camden_Council_(New_South_Wales),34°03′S,150°42′E,Camden Council,-34.050000,150.700000
9,/wiki/City_of_Campbelltown_(New_South_Wales),34°04′S,150°49′E,City of Campbelltown,-34.066667,150.816667
...,...,...,...,...,...,...
726,/wiki/City_of_Hawkesbury,33°25′S,150°47′E,Hawkesbury,-33.416667,150.783333
727,/wiki/The_Hills_Shire,33°46′S,151°00′E,The Hills,-33.766667,151.000000
728,/wiki/Municipality_of_Hunter%27s_Hill,33°49′S,151°08′E,Hunter's Hill,-33.816667,151.133333
729,/wiki/Ku-ring-gai_Council,33°45′15″S,151°09′06″E,Ku-ring-gai,-33.754167,151.151667


In [31]:
def venue_by_suburb(sub, sub_lat, sub_lng):
    r = requests.get(f"{venue_explore_url}&ll={sub_lat},{sub_lng}&radius=1000&limit=50")
    j = r.json()
    print(j["response"].keys())
    if not 'groups' in j['response'] or len(j['response']['groups']) == 0:
        print("haha")
        return None
    items = j['response']['groups'][0]['items']
    names = []
    lats = []
    lngs = []
    cats = []

    for i in items:
        names.append(i['venue']['name'])
        lats.append(i['venue']['location']['lat'])
        lngs.append(i['venue']['location']['lng'])
        if len(i['venue']['categories']) == 0:
            cats.append('None')
        else:
            cats.append(i['venue']['categories'][0]['name'])
    df = pd.DataFrame({'Suburb': sub, 'Suburb_Latitude': sub_lat, 'Suburb_Longitude': sub_lng, 'Venue' : names, 'Venue_Latitude' : lats, 'Venue_Longitude': lngs, 'Venue_Category': cats})
    return df 


In [32]:
df = []
for _, row in tqdm(suburb.iterrows()):
    df.append(venue_by_suburb(row['suburb'], row['lat_dec'], row['lng_dec']))

1it [00:01,  1.25s/it]dict_keys([])
haha
2it [00:01,  1.01s/it]dict_keys([])
haha
3it [00:02,  1.18it/s]dict_keys([])
haha
3it [00:03,  1.10s/it]


KeyboardInterrupt: 

In [39]:
j = requests.get(f"{venue_explore_url}&ll={30},{30}&radius=1000&limit=50")

In [42]:
j.text

'{"meta":{"code":429,"errorType":"quota_exceeded","errorDetail":"Quota exceeded","requestId":"601f7c85db8c8f201f8bc5e6"},"response":{}}'